<a href="https://colab.research.google.com/github/papasanimohansrinivas/IndianAccentSpeechRecognition/blob/master/PytorchImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/papasanimohansrinivas/IndianAccentSpeechRecognition.git

Cloning into 'IndianAccentSpeechRecognition'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 10 (delta 4), reused 10 (delta 4), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [0]:
ls /content

IndianAccentSpeechRecognition/  sample_data/


In [0]:
cp IndianAccentSpeechRecognition/installation_script.sh /content/

In [0]:
cp IndianAccentSpeechRecognition/download_script.sh /content/

In [0]:
cp IndianAccentSpeechRecognition/download_pretrainedmodels.sh /content/

In [0]:
!sudo bash ./download_pretrainedmodels.sh
!sudo bash ./installation_script.sh
!sudo bash ./download_script.sh

In [0]:
from pandas import read_csv
from os import path
import re

In [0]:
tr,dv=map(read_csv,["/content/train.csv","/content/dev.csv"])

In [0]:
tr.wav_filename=tr.wav_filename.apply(lambda txt : path.join('/content/train_wav_files/',txt.split("/")[-1]))
dv.wav_filename=dv.wav_filename.apply(lambda txt : path.join('/content/dev_wav_files/',txt.split("/")[-1]))

In [0]:
dv.transcript = dv.transcript.str.encode("utf-8").apply(lambda txt : re.sub("[^A-Z ]",'',txt.decode("ascii","ignore").strip().upper()))
tr.transcript = tr.transcript.str.encode("utf-8").apply(lambda txt : re.sub("[^A-Z ]",'',txt.decode("ascii","ignore").strip().upper()))

In [0]:
dv['identity'],tr['identity']=1,0

In [0]:
def write_to_txt(row):
  if row['identity'] == 0 :
    folder = '/content/processed_train_txt_files/'
  else:
    folder = '/content/processed_dev_txt_files/'   
    
  with open(path.join(folder,row['wav_filename'].split("/")[-1].replace(".wav",".txt")),"w") as txtfile:
    txtfile.write(row['transcript'])

In [0]:
dnone = tr.apply(write_to_txt,axis=1)
trnone = dv.apply(write_to_txt ,axis=1)

In [0]:
tr['txt_path'] = tr['wav_filename'].apply(lambda txt : path.join("/content/processed_train_txt_files/",txt.split("/")[-1].replace(".wav",".txt")))
dv['txt_path'] = dv['wav_filename'].apply(lambda txt : path.join("/content/processed_dev_txt_files/",txt.split("/")[-1].replace(".wav",".txt")))

In [0]:
dv[['wav_filename','txt_path']].to_csv("/content/processed_dev.csv",header=False,index=False)
tr[['wav_filename','txt_path']].to_csv("/content/processed_train.csv",header=False,index=False)

In [0]:
cd /content/

In [0]:
ls

benchmark.py  labels.json  multiproc.py     README.md         train.py
data/         LICENCE      noise_inject.py  requirements.txt  transcribe.py
decoder.py    logger.py    opts.py          server.py         tune_decoder.py
Dockerfile    model.py     __pycache__/     test.py           utils.py


In [0]:
cd deepspeech.pytorch/

/content/deepspeech.pytorch


In [0]:
# !python ./train.py --help

In [0]:
# rm -rf /content/pytorch_speech_model/*

In [0]:
# --continue-from /content/pretrained_speech_model/librispeech_pretrained.pth --finetune

In [0]:
!python ./train.py  --train-manifest /content/processed_train.csv  --val-manifest  /content/processed_dev.csv  --rnn-type gru --cuda   --epochs 2 --batch-size 45 --save-folder /content/pytorch_speech_model/ --checkpoint --checkpoint-per-batch 1  --hidden-layers 3 --hidden-size 132 --labels-path /content/deepspeech.pytorch/labels.json --learning-anneal 0.01

DeepSpeech(
  (conv): MaskConv(
    (seq_module): Sequential(
      (0): Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2), padding=(20, 5))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=0, max_val=20, inplace)
      (3): Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1), padding=(10, 5))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Hardtanh(min_val=0, max_val=20, inplace)
    )
  )
  (rnns): Sequential(
    (0): BatchRNN(
      (rnn): GRU(1312, 132, bidirectional=True)
    )
    (1): BatchRNN(
      (batch_norm): SequenceWise (
      BatchNorm1d(132, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
      (rnn): GRU(132, 132, bidirectional=True)
    )
    (2): BatchRNN(
      (batch_norm): SequenceWise (
      BatchNorm1d(132, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))
      (rnn): GRU(132, 132, bidirectional=True)
    

In [0]:
from google.colab import files

In [0]:
# ls /content/train_wav_files/

In [0]:
rr=files.upload()

Saving Rasa__0.wav to Rasa__0.wav


In [0]:
!python transcribe.py --model-path ./models/deepspeech_final.pth --audio-path 'Rasa_0.wav' --cuda

{"output": [{"transcription": "IM"}], "_meta": {"acoustic_model": {"name": "deepspeech_final.pth"}, "language_model": {"name": null}, "decoder": {"lm": false, "alpha": null, "beta": null, "type": "greedy"}}}


In [0]:
"II'IXI".encode("utf-8")

b"II'IXI"

In [0]:
!python transcribe.py --model-path ./models/deepspeech_final.pth --audio-path '/content/train_wav_files/00076348e0aa9b2d2a707a9d8559087e3e90920d726d725ab2ee78139cae39c3938bfd3ce40dac74caa54652b4b9ebc47a2ff0e2bf88b9e75abd03f848a5807b.wav' --cuda

{"output": [{"transcription": ""}], "_meta": {"acoustic_model": {"name": "deepspeech_final.pth"}, "language_model": {"name": null}, "decoder": {"lm": false, "alpha": null, "beta": null, "type": "greedy"}}}


In [0]:
files.download("/content/pytorch_speech_model/deepspeech_1.pth.tar")
files.download("./models/deepspeech_final.pth")